Imports
=======

``` ipython
import torch
import torch.nn as nn
import torch.optim as optim
import torchmetrics
from torch.utils.data import Dataset, TensorDataset, DataLoader

REPO_ROOT = "/home/leon/models/NeuroFlame"

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pal = sns.color_palette("tab10")
DEVICE = 'cuda:1'
```

``` ipython
import sys
sys.path.insert(0, '../../../')

from notebooks.setup import *

import pandas as pd
import torch.nn as nn
from time import perf_counter
from scipy.stats import circmean

from src.network import Network
from src.plot_utils import plot_con
from src.decode import decode_bump, circcvl
from src.lr_utils import masked_normalize, clamp_tensor, normalize_tensor
```

Notebook Settings
=================

``` ipython
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%run ../../../notebooks/setup.py
%matplotlib inline
%config InlineBackend.figure_format = 'png'
```

Helpers
=======

Other
-----

``` ipython
def calculate_mean_accuracy_and_sem(accuracies):
    mean_accuracy = accuracies.mean()
    std_dev = accuracies.std(unbiased=True).item()
    sem = std_dev / np.sqrt(len(accuracies))
    return mean_accuracy, sem
```

``` ipython
def angle_AB(A, B):
      A_norm = A / (np.linalg.norm(A) + 1e-5)
      B_norm = B / (np.linalg.norm(B) + 1e-5)

      return int(np.arccos(A_norm @ B_norm) * 180 / np.pi)
```

``` ipython
def get_theta(a, b, GM=0, IF_NORM=0):

      u, v = a, b

      if GM:
          v = b - np.dot(b, a) / np.dot(a, a) * a

      if IF_NORM:
          u = a / np.linalg.norm(a)
          v = b / np.linalg.norm(b)

      return np.arctan2(v, u) % (2.0 * np.pi)
```

``` ipython
def get_idx(model, rank=1):
      # print(model.low_rank.U.shape)
      # ksi = torch.vstack((model.low_rank.U[:,0], model.low_rank.U[:,1]))
      ksi = torch.hstack((model.low_rank.V, model.low_rank.U)).T
      ksi = ksi[:, :model.Na[0]]

      try:
            readout = model.low_rank.linear.weight.data
            ksi = torch.vstack((ksi, readout))
      except:
            pass

      print('ksi', ksi.shape)

      ksi = ksi.cpu().detach().numpy()
      theta = get_theta(ksi[0], ksi[rank])

      return theta.argsort()
```

``` ipython
def get_overlap(model, rates):
      ksi = model.odors.cpu().detach().numpy()
      return rates @ ksi.T / rates.shape[-1]
```

``` ipython
import scipy.stats as stats

def plot_smooth(data, ax, color):
      mean = data.mean(axis=0)
      ci = smooth.std(axis=0, ddof=1) * 1.96

      # Plot
      ax.plot(mean, color=color)
      ax.fill_between(range(data.shape[1]), mean - ci, mean + ci, alpha=0.25, color=color)

```

``` ipython
def convert_seconds(seconds):
      h = seconds // 3600
      m = (seconds % 3600) // 60
      s = seconds % 60
      return h, m, s
```

plots
-----

``` ipython
import pickle as pkl
import os
def pkl_save(obj, name, path="."):
    os.makedirs(path, exist_ok=True)
    destination = path + "/" + name + ".pkl"
    print("saving to", destination)
    pkl.dump(obj, open(destination, "wb"))


def pkl_load(name, path="."):
    source = path + "/" + name + '.pkl'
    # print('loading from', source)
    return pkl.load(open( source, "rb"))

```

``` ipython
def add_vlines(ax=None, mouse=""):
    t_BL = [0, 1]
    t_STIM = [1 , 2]
    t_ED = [2, 3]
    t_DIST = [3 , 4]
    t_MD = [4 , 5]
    t_CUE = [5 , 5.5]
    t_RWD = [5.5, 6.0]
    t_LD = [6.0 , 7.0]
    t_TEST = [7.0, 8.0]
    t_RWD2 = [11 , 12]

    # time_periods = [t_STIM, t_DIST, t_TEST, t_CUE, t_RWD, t_RWD2]
    # colors = ["b", "b", "b", "g", "y", "y"]

    time_periods = [t_STIM, t_DIST, t_TEST, t_CUE]
    colors = ["b", "b", "b", "g"]

    if ax is None:
        for period, color in zip(time_periods, colors):
            plt.axvspan(period[0], period[1], alpha=0.1, color=color)
    else:
        for period, color in zip(time_periods, colors):
            ax.axvspan(period[0], period[1], alpha=0.1, color=color)

```

``` ipython
def add_vdashlines(ax=None, mouse=""):
    t_BL = [0, 1]
    t_STIM = [1 , 2]
    t_ED = [2, 3]
    t_DIST = [3 , 4]
    t_MD = [4 , 5]
    t_CUE = [5 , 5.5]
    t_RWD = [5.5, 6.0]
    t_LD = [6.0 , 7.0]
    t_TEST = [7.0, 8.0]
    t_RWD2 = [11 , 12]

    # time_periods = [t_STIM, t_DIST, t_TEST, t_CUE, t_RWD, t_RWD2]
    # colors = ["b", "b", "b", "g", "y", "y"]

    time_periods = [t_STIM, t_DIST, t_TEST, t_CUE]
    colors = ["b", "b", "b", "g"]

    if ax is None:
        for period, color in zip(time_periods, colors):
            plt.axvline(period[0], ls='--', color=color)
            plt.axvline(period[1], ls='--', color=color)
    else:
        for period, color in zip(time_periods, colors):
            ax.axvline(period[0], ls='--', color=color)
            ax.axvline(period[1], ls='--', color=color)
```

``` ipython
def plot_rates_selec(rates, idx, thresh=0.5, figname='fig.svg'):
        ordered = rates[..., idx]
        fig, ax = plt.subplots(1, 2, figsize=[2*width, height])
        r_max = thresh * np.max(rates[0])

        ax[0].imshow(rates[0].T, aspect='auto', cmap='jet', vmin=0, vmax=r_max)
        ax[0].set_ylabel('Neuron #')
        ax[0].set_xlabel('Step')

        ax[1].imshow(ordered[0].T, aspect='auto', cmap='jet', vmin=0, vmax=r_max)
        ax[1].set_yticks(np.linspace(0, model.Na[0].cpu().detach(), 5), np.linspace(0, 360, 5).astype(int))
        ax[1].set_ylabel('Pref. Location (°)')
        ax[1].set_xlabel('Step')
        plt.savefig(figname, dpi=300)
        plt.show()
```

``` ipython
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import sem, t

def mean_ci(data):
  # Calculate mean and SEM
  mean = np.nanmean(data, axis=0)
  serr = sem(data, axis=0, nan_policy='omit')

  # Calculate the t critical value for 95% CI
  n = np.sum(~np.isnan(data), axis=0)
  t_val = t.ppf(0.975, df=n - 1)  # 0.975 for two-tailed 95% CI

  # Calculate 95% confidence intervals
  ci = t_val * serr

  return mean, ci

def plot_overlap_dpa(readout, y, axis=0, label=['pair', 'unpair'], figname='fig.svg', title='first'):

    fig, ax = plt.subplots(1, 2, figsize=[2*width, height], sharey=True)

    time = np.linspace(0, 10, readout.shape[1])
    trial = [0, 1]
    colors = ['r', 'b', 'g']
    ls = ['--', '-']

    for j in range(2): # pair unpair
        for i in range(2):
            # if axis == 0:
            #     sign_readout = 2*y[-1, :, np.newaxis, np.newaxis] * readout
            #     data = sign_readout[(y[0]==i) & (y[1]==trial[j]), :, axis]
            # else:
            data = readout[(y[0]==i) & (y[1]==trial[j]), :, axis]

            mean, ci = mean_ci(data)
            ax[j].plot(time, mean, ls=ls[i], label=label[i], color=colors[j])
            ax[j].fill_between(time, mean - ci, mean + ci, color=colors[j], alpha=0.1)

        add_vlines(ax[j])
        ax[j].set_xlabel('Time (s)')

        if axis==0:
            ax[j].set_ylabel('A/B Overlap (Hz)')
        elif axis==1:
            ax[j].set_ylabel('GNG Overlap (Hz)')
        else:
            ax[j].set_ylabel('Readout (Hz)')

        ax[j].axhline(0, color='k', ls='--')

    plt.savefig('../figures/dual/%s' % figname, dpi=300)
    plt.show()
```

``` ipython
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import sem, t

def mean_ci(data):
  # Calculate mean and SEM
  mean = np.nanmean(data, axis=0)
  serr = sem(data, axis=0, nan_policy='omit')

  # Calculate the t critical value for 95% CI
  n = np.sum(~np.isnan(data), axis=0)
  t_val = t.ppf(0.975, df=n - 1)  # 0.975 for two-tailed 95% CI

  # Calculate 95% confidence intervals
  ci = t_val * serr

  return mean, ci
```

``` ipython
def plot_avg_overlap(readout, n_batch, labels=['A', 'B'], figname='fig.svg'):
      fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

      time = np.linspace(0, 10, readout.shape[1])
      size = readout.shape[0] // 2
      print(readout.shape[0], size)

      readout = readout.reshape((3, ))

      for i in range(readout.shape[-1]):
            if i==0:
                  ax[i].plot(time, (readout[:size, :, i].T - readout[size:,:,i].T), ls='-', label=labels[0])
            else:
                  ax[i].plot(time, readout[size:, :, i].T, ls='--', label='Go')

            add_vlines(ax[i])
            ax[i].set_xlabel('Time (s)')

      ax[0].set_ylabel('Sample Overlap (Hz)')
      ax[1].set_ylabel('Go/NoGo Overlap (Hz)')
      # ax[2].set_ylabel('Readout (Hz)')

      # plt.legend(fontsize=10, frameon=False)
      plt.savefig(figname, dpi=300)
      plt.show()
```

``` ipython
def plot_m0_m1_phi(rates, idx, figname='fig.svg'):

      m0, m1, phi = decode_bump(rates[..., idx], axis=-1)
      fig, ax = plt.subplots(1, 3, figsize=[3*width, height])

      time = np.linspace(0, 10, m0.T.shape[0])

      ax[0].plot(time, m0[:2].T)
      ax[0].plot(time, m0[2:].T, '--')
      #ax[0].set_ylim([0, 360])
      #ax[0].set_yticks([0, 90, 180, 270, 360])
      ax[0].set_ylabel('$\mathcal{F}_0$ (Hz)')
      ax[0].set_ylabel('Activity (Hz)')
      ax[0].set_xlabel('Time (s)')
      add_vlines(ax[0])

      ax[1].plot(time, m1[:2].T)
      ax[1].plot(time, m1[2:].T, '--')
      # ax[1].set_ylim([0, 360])
      # ax[1].set_yticks([0, 90, 180, 270, 360])
      ax[1].set_ylabel('$\mathcal{F}_1$ (Hz)')
      ax[1].set_ylabel('Bump Amplitude (Hz)')
      ax[1].set_xlabel('Time (s)')
      add_vlines(ax[1])

      ax[2].plot(time, phi[:2].T * 180 / np.pi)
      ax[2].plot(time, phi[2:].T * 180 / np.pi, '--')
      ax[2].set_ylim([0, 360])
      ax[2].set_yticks([0, 90, 180, 270, 360])
      ax[2].set_ylabel('Bump Center (°)')
      ax[2].set_xlabel('Time (s)')
      add_vlines(ax[2])

      plt.savefig(figname, dpi=300)
      plt.show()
```

``` ipython
from matplotlib.patches import Circle

def plot_fix_points(rates, ax, title='', color='k'):
    m0, m1, phi = decode_bump(rates[:, -1], axis=-1)

    x = np.cos(phi)
    y = np.sin(phi)

    xNoGo = np.cos(3*np.pi /2.)
    yNoGo = np.sin(3*np.pi /2)

    xGo = np.cos(np.pi /2.)
    yGo = np.sin(np.pi /2)

    # rad = np.max(np.sqrt(x**2+y**2))

    ax.plot(x, y, 'o', ms=15, color=color)
    ax.plot(xGo, yGo, 'o', ms=15, color='w', markeredgecolor='k')
    ax.plot(xNoGo, yNoGo, 'o', ms=15, color='w', markeredgecolor='k')
    circle = Circle((0., 0.), 1, fill=False, edgecolor='k')
    ax.add_patch(circle)

    # Set the aspect of the plot to equal to make the circle circular
    ax.set_aspect('equal')
    ax.set_title(title)
    ax.axis('off')
    # plt.savefig('fp_dpa.svg', dpi=300)
    # plt.show()
```

``` ipython
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Define custom colormap with red at the center
cdict = {
    'red':   [(0.0, 0.0, 0.0),
              (0.5, 1.0, 1.0),
              (1.0, 1.0, 1.0)],
    'green': [(0.0, 0.0, 0.0),
              (0.5, 0.0, 0.0),
              (1.0, 1.0, 1.0)],
    'blue':  [(0.0, 1.0, 1.0),
              (0.5, 0.0, 0.0),
              (1.0, 0.0, 0.0)]
}

custom_cmap = LinearSegmentedColormap('RedCenterMap', cdict)

# Plot to visualize the colormap
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

fig, ax = plt.subplots(figsize=(6, 1))
ax.imshow(gradient, aspect='auto', cmap=custom_cmap)
ax.set_axis_off()
plt.show()
```

``` ipython
def plot_overlap(readout, labels=['pair', 'unpair'], figname='fig.svg'):
      fig, ax = plt.subplots(1, readout.shape[-1], figsize=[readout.shape[-1]*width, height])

      time = np.linspace(0, 10, readout.shape[1])
      size = readout.shape[0] // 2

      for i in range(readout.shape[-1]):
            ax[i].plot(time, readout[:size, :, i].T, ls='-', label=labels[0])
            if i==0:
                  ax[i].plot(time, -readout[size:, :, i].T, ls='--', label=labels[1])
            else:
                  ax[i].plot(time, readout[size:, :, i].T, ls='--', label=labels[1])

            add_vlines(ax[i])
            ax[i].set_xlabel('Time (s)')

      ax[0].set_ylabel('Sample Overlap (Hz)')
      ax[1].set_ylabel('Choice Overlap (Hz)')
      if readout.shape[-1] == 3:
            ax[-1].set_ylabel('Readout (Hz)')

      # ax[1].legend(fontsize=10, frameon=False)
      plt.savefig(figname, dpi=300)
      plt.show()
```

Model
=====

``` ipython
kwargs = {} # {'VAR_FF': [.25, .25]}
```

``` ipython
REPO_ROOT = "/home/leon/models/NeuroFlame"
conf_name = "train_dual.yml"
DEVICE = 'cuda:1'

seed = np.random.randint(0, 100)
# seed = 3
print(seed)


```

``` ipython
model = Network(conf_name, REPO_ROOT, VERBOSE=0, DEVICE=DEVICE, SEED=seed, N_BATCH=1, **kwargs);
path = model.SAVE_PATH
device = torch.device(DEVICE if torch.cuda.is_available() else 'cpu')
model.to(device)
```

``` ipython
print(seed)
```

``` ipython
import gc
gc.collect()

torch.cuda.empty_cache()
torch.cuda.device(DEVICE)   # where X is the GPU index, e.g., 0, 1
torch.cuda.synchronize()
torch.cuda.reset_accumulated_memory_stats(DEVICE)
```

DPA Task
========

Testing
-------

``` ipython
model_state_dict = torch.load('%s/dpa_%d.pth' % (path, seed))
model.load_state_dict(model_state_dict, strict=False)
```

``` ipython
print(model.J_STP)
```

``` ipython
steps = np.arange(0, model.N_STEPS - model.N_STEADY, model.N_WINDOW)

mask = (steps >= (model.N_STIM_OFF[0].cpu().numpy() - model.N_STEADY)) & (steps <= (model.N_STEPS - model.N_STEADY))
rwd_idx = np.where(mask)[0]
print('rwd', rwd_idx)

model.lr_eval_win = rwd_idx.shape[0]

stim_mask = (steps >= (model.N_STIM_ON[0].cpu().numpy() - model.N_STEADY)) & (steps < (model.N_STIM_OFF[0].cpu().numpy() - model.N_STEADY))
stim_idx = np.where(stim_mask)[0]
print('stim', stim_idx)

test_mask = (steps >= (model.N_STIM_ON[-1].cpu().numpy() - model.N_STEADY)) & (steps < (model.N_STIM_OFF[-1].cpu().numpy() - model.N_STEADY))
test_idx = np.where(test_mask)[0]
print('test', test_idx)

zero_idx = np.where(~mask & ~stim_mask )[0]
print('zero', zero_idx)
```

``` ipython
N_BATCH = 256
model.N_BATCH = N_BATCH

ff_input = []

# 2 readouts (sample/choice), 4 conditions AC, AD, BC, BD
y_labels = np.zeros((3, 4, model.N_BATCH))

l=0
for i in [-1, 1]:
        for k in [-1, 1]:

            model.I0[0] = i # sample
            model.I0[4] = k # test

            if k == 1: # test
                    y_labels[2, l] = np.ones(model.N_BATCH)

            if i == 1: # sample
                    y_labels[1, l] = np.ones(model.N_BATCH)

            if i==k: # Pair Trials
                    y_labels[0, l] = np.ones(model.N_BATCH)

            l+=1
            ff_input.append(model.init_ff_input())

y_labels = torch.tensor(y_labels, dtype=torch.float, device=DEVICE).reshape(3, -1)

ff_input = torch.vstack(ff_input)
print('ff_input', ff_input.shape, 'y_labels', y_labels.shape)
```

``` ipython
with torch.no_grad():
    rates = model.forward(ff_input=ff_input).detach().cpu().numpy()
print(rates.shape)
```

``` ipython
pkl_save(rates, 'dpa_rates')
pkl_save(y_labels, 'dpa_labels')
```

``` ipython
import numpy as np

X1 = model.odors[4].cpu().numpy()  # class 1 samples
X2 = -model.odors[9].cpu().numpy()  # class 2 samples

w = X1 - X2            # normal vector to the hyperplane
c = 0.5 * np.dot(w, X1 + X2)  # offset

def predict(x):
    return (x @ w ) / w.shape[0] # >0: side of X1, <0: side of X2
```

``` ipython
dpa_pred = predict(rates)
print(dpa_pred.shape)
```

``` ipython
dpa_labels = y_labels.cpu().numpy()
print(dpa_labels.shape)

idxPair = (dpa_labels[0]==1)
idxUnpair = (dpa_labels[0]==0)
```

``` ipython
plt.plot(np.linspace(0, 10, 101), ((2 * dpa_labels[0]-1)[idxPair, None] * dpa_pred[idxPair]).mean(0), label='Pair', color='r')
plt.plot(np.linspace(0, 10, 101), ((2 * dpa_labels[0]-1)[idxUnpair, None] * dpa_pred[idxUnpair]).mean(0), label='Unpair', color='r', ls='--')

add_vlines()
plt.xlabel('Time')
plt.ylabel('Test Overlap')
plt.ylim([-1, 1])
plt.axhline(0, ls='--')
plt.legend(fontsize=12, frameon=0)
plt.savefig('../figures/bernstein/dpa_test_overlap.svg', dpi=300)
plt.show()
```

``` ipython
def plot_overlap_label(readout, y, task=0, label=['pair', 'unpair'], figname='fig.svg', title='first'):
    '''
    y[0] is pairs, y[1] is samples, y[2] is task if not None
    '''
    print(y.shape)
    size = y.shape[0]
    if size ==2:
        ones_slice = np.zeros(y.shape)
        y_ = np.vstack((y.copy(), ones_slice))
        task = 0
    else:
        y_ = y.copy()

    print(y_.shape)
    fig, ax = plt.subplots(1, size+1, figsize=[(size+1)*width, height])

    time = np.linspace(0, 10, readout.shape[1])
    colors = ['r', 'b', 'g']
    ls = ['--', '-', '--', '-']
    label = ['BC', 'BD', 'AD', 'AC']

    mean_overlaps = []
    for k in range(2): # readout
        for j in range(2): # sample
            for i in range(2): # pair
                data = readout[(y_[0]==i) & (y_[1]==j) & (y_[2]==task), :, k]

                mean, ci = mean_ci(data)

                mean_overlaps.append(mean)

                ax[k].plot(time, mean, ls=ls[i+2*j], label=label[i+2*j], color=colors[task], alpha=1-(1-j)/2)
                ax[k].fill_between(time, mean - ci, mean + ci, color=colors[task], alpha=0.1)

                add_vlines(ax[k])
                ax[k].set_xlabel('Time (s)')

        if k==0:
            ax[k].set_ylabel('A/B Overlap (Hz)')
        elif k==1:
            ax[k].set_ylabel('Choice Overlap (Hz)')
        else:
            ax[k].set_ylabel('Readout (Hz)')

        ax[k].axhline(0, color='k', ls='--')

    mean_overlaps = np.array(mean_overlaps).reshape((2, 2, 2, -1))
    print(mean_overlaps.shape)
    for j in range(2): # sample
        for i in range(2): # pair
            ax[-1].plot(mean_overlaps[0, j, i], mean_overlaps[1, j, i], color=colors[task], ls=ls[i+2*j], label=label[i+2*j], alpha=1-(1-j)/2)

    ax[-1].set_xlabel('A/B Overlap (Hz)')
    ax[-1].set_ylabel('Choice Overlap (Hz)')

    plt.legend(fontsize=10)
    plt.savefig('../figures/dual/%s' % figname, dpi=300)
    plt.show()
```

``` ipython
plot_overlap_label(model.readout.cpu().detach().numpy(), y_labels.cpu().numpy()[:2], task=0, figname='dpa_overlaps.svg')
```

``` ipython
U = model.low_rank.U.cpu().detach().numpy()
V = model.low_rank.V.cpu().detach().numpy()

fig, ax = plt.subplots(1, 2, figsize=[2*width, height])
# ax[0].hist(U[:, 0], histtype='step', bins='auto')
# ax[0].hist(U[:, 1], histtype='step', bins='auto')
ax[0].hist(V[:, 0], histtype='step', bins='auto')
ax[1].hist(V[:, 1], histtype='step', bins='auto')
ax[0].set_xlabel('$ n_{AB} $')
ax[1].set_xlabel('$ n_{GNG} $')

ax[0].set_ylabel('Count')
ax[1].set_ylabel('Count')
plt.show()
```

``` ipython
odors = model.odors.cpu().numpy()

m1 = U[:, 0]
n1 = V[:, 0]

m2 = U[:, 1]
n2 = V[:, 1]

vectors = [m1 * m2, n1 * n2, odors[4], -odors[9]]
# labels = ['$m_\\text{AB}$', '$n_\\text{AB}$', '$m_\\text{GnG}$', '$n_\\text{GnG}$', 'C', 'D']
labels = ['$m_\\text{AB}$ $m_\\text{GnG}$', '$n_\\text{AB}$ $n_\\text{GnG}$', 'odor C', 'odor D']
```

``` ipython
import numpy as np
import matplotlib.pyplot as plt

num_vectors = len(vectors)
cov_matrix = np.zeros((num_vectors, num_vectors))

for i in range(num_vectors):
    for j in range(num_vectors):
        cov_matrix[i][j] = angle_AB(vectors[i], vectors[j])

# Mask the upper triangle
mask = np.triu(np.ones_like(cov_matrix, dtype=bool))
masked_cov_matrix = np.ma.masked_array(cov_matrix, mask=mask)
```

``` ipython
def plot_cov(mask, num_vectors, labels):
    # Plot the masked covariance matrix
    img = plt.imshow(mask, cmap='coolwarm', interpolation='nearest')
    # cbar = plt.colorbar(label='Angle (°)')
    # cbar.set_ticks([30, 90, 120])

    # Set axis labels on top and left
    plt.xticks(ticks=np.arange(num_vectors), labels=labels, fontsize=18)
    plt.yticks(ticks=np.arange(num_vectors), labels=labels, fontsize=18)

    # Invert y-axis
    ax.xaxis.set_ticks_position('top')
    ax.xaxis.set_label_position('top')

    # ax.yaxis.set_ticks_position('right')
    # ax.yaxis.set_label_position('right')
    ax.invert_yaxis()

    for i in range(num_vectors):
        for j in range(i + 1):
            plt.text(j, i, f'{cov_matrix[i, j]:.0f}', ha='center', va='center', color='black')
```

``` ipython
fig, ax = plt.subplots(figsize=(8, 6))
plot_cov(masked_cov_matrix, num_vectors, labels)
# plt.savefig('../figures/dual/cov_dpa_%d.svg' % seed, dpi=300)
plt.show()
```

``` ipython
U = model.low_rank.U.cpu().detach().numpy()
V = model.low_rank.V.cpu().detach().numpy()

fig, ax = plt.subplots(1, 2, figsize=[2*width, height])
# ax[0].hist(U[:, 0], histtype='step', bins='auto')
# ax[0].hist(U[:, 1], histtype='step', bins='auto')
ax[0].hist(V[:, 0], histtype='step', bins='auto')
ax[1].hist(V[:, 1], histtype='step', bins='auto')
ax[0].set_xlabel('$ n_{AB} $')
ax[1].set_xlabel('$ n_{GNG} $')

ax[0].set_ylabel('Count')
ax[1].set_ylabel('Count')
plt.savefig('lr_hist_dpa.svg', dpi=300)
plt.show()
```

``` ipython
U = model.low_rank.U.cpu().detach().numpy()
V = model.low_rank.V.cpu().detach().numpy()

fig, ax = plt.subplots(1, 4, figsize=[4*height, height])

ax[0].scatter(U[:, 0], U[:, 1])
ax[0].set_xlabel('$ m_{AB} $')
ax[0].set_ylabel('$ m_{Choice} $')

ax[1].scatter(V[:, 0], V[:, 1])
ax[1].set_xlabel('$ n_{AB} $')
ax[1].set_ylabel('$ n_{Choice} $')

ax[2].scatter(U[:, 0], V[:, 0])
ax[2].set_xlabel('$ m_{AB} $')
ax[2].set_ylabel('$ n_{AB} $')

ax[3].scatter(U[:, 1], V[:, 1])
ax[3].set_xlabel('$ m_{Choice} $')
ax[3].set_ylabel('$ n_{Choice} $')

plt.savefig('lr_scatter_dpa.svg', dpi=300)
plt.show()
```

``` ipython
theta = np.arctan2(V[:, 1], V[:, 0])
idx = np.argsort(theta)
print(idx.shape)
plt.hist(theta * 180 / np.pi)
plt.show()
```

``` ipython
from src.lr_utils import clamp_tensor

Wab = (model.W_stp_T[0] + model.Wab_train / model.Na[0])
Wab = clamp_tensor(Wab, 0, model.slices)

# Wab = model.Wab_train / model.Na[0]

Wab = Wab.detach().cpu().numpy()
Wab_bin = Wab>0

print(Wab.shape)
print(Wab[0,: 10])
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

ax[0].imshow(Wab[idx][idx], cmap='jet', vmin=0, vmax=.05)
ax[1].imshow(Wab_bin[idx][idx], cmap='jet', vmin=0, vmax=1)

plt.show()
```

``` ipython
plt.figure(figsize=(2.5*width, 1.5*height))  # Set the figure size (width, height) in inches
Cij = Wab_bin[idx][idx]
ax1 = plt.subplot2grid((2, 3), (0, 0), rowspan=2)
im = ax1.imshow(Cij, cmap='jet', aspect=1, vmin=0)
ax1.set_xlabel("Presynaptic")
ax1.set_ylabel("Postsynaptic")

# Second column, first row
ax2 = plt.subplot2grid((2, 3), (0, 1))
Kj = np.sum(Cij, axis=0)  # sum over pres
ax2.plot(circcvl(Kj, windowSize=75))
# ax2.set_xticklabels([])
ax2.set_ylabel("$K_j$")

# # Second column, second row
ax3 = plt.subplot2grid((2, 3), (1, 1))
Ki = np.sum(Cij, axis=1)  # sum over pres
ax3.plot(circcvl(Ki, windowSize=75))
ax3.set_ylabel("$K_i$")

ax4 = plt.subplot2grid((2, 3), (0, 2), rowspan=2)
diags = []
for i in range(int(Cij.shape[0] / 2)):
    diags.append(np.trace(Cij, offset=i) / Cij.shape[0])
diags = np.array(diags)
ax4.plot(diags)
ax4.set_xlabel("Neuron #")
ax4.set_ylabel("$P_{ij}$")

plt.tight_layout()
plt.show()
```

``` ipython
print(dpa_labels.shape, rates.shape)
```

``` ipython
fig, ax = plt.subplots(2, 2, figsize=[2*width, 2*height])

for j in range(2):
    for i in range(2):
        trial_idx = np.argwhere((dpa_labels[-1] == j) & (dpa_labels[1]==1-i))[0]
        k = np.random.randint(len(trial_idx))

        vmin, vmax = np.percentile(rates[trial_idx[k], :, idx].reshape(-1), [5, 95])
        ax[j][i].imshow(rates[trial_idx[k], :, idx], aspect='auto', cmap='viridis', vmin=vmin, vmax=vmax, extent=[0, 10, 0, 360])

        ax[j][i].set_yticks(np.linspace(0, 360, 5))
        if i==0:
            ax[j][i].set_ylabel('Pref. Loc. (°)')
        if j==1:
            ax[j][i].set_xlabel('Time (s)')

# add_vdashlines()
plt.savefig('../figures/bernstein/raster_AD.svg', dpi=300)
plt.show()
```

``` ipython
from scipy.ndimage import gaussian_filter1d, uniform_filter1d
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

ax[0].plot(np.linspace(0, 360, 750), uniform_filter1d(model.odors[0][idx].cpu(), size=75), label='A', color='r')
ax[0].plot(np.linspace(0, 360, 750), uniform_filter1d(-model.odors[5][idx].cpu(), size=75), label='B', color='k')


ax[0].set_xlabel('Pref Loc. (°)')
ax[0].set_ylabel('Sample Input')
ax[0].set_xticks(np.linspace(0, 360, 5))
ax[0].legend(fontsize=12, frameon=False)

ax[1].plot(np.linspace(0, 360, 750), uniform_filter1d(model.odors[4][idx].cpu(), size=75), label='C')
ax[1].plot(np.linspace(0, 360, 750), uniform_filter1d(-model.odors[9][idx].cpu(), size=75), label='D')

ax[1].set_xlabel('Pref Loc. (°)')
ax[1].set_ylabel('Test Input')
ax[1].set_xticks(np.linspace(0, 360, 5))
ax[1].legend(fontsize=12, frameon=False)

ax[1].set_ylim([-0.75, 1.25])
ax[0].set_ylim([-0.75, 1.25])

plt.savefig('../figures/bernstein/dpa_inputs.svg', dpi=300)

plt.show()
```

``` ipython
from scipy.ndimage import gaussian_filter1d, uniform_filter1d
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

w = model.odors[0].cpu().numpy() + model.odors[1].cpu().numpy()
ax[0].plot(np.linspace(0, 360, 750), uniform_filter1d(w[idx], size=75))
ax[0].set_ylabel("Weights A/B")
# ax[0].axvline(180, ls='--', color='k')

w = model.odors[4].cpu().numpy() + model.odors[9].cpu().numpy()

ax[1].plot(np.linspace(0, 360, 750), uniform_filter1d(w[idx], size=75))
ax[1].set_ylabel("Weights C/D")

# ax[1].axvline(45, ls='--', color='k')
# ax[1].axvline(225, ls='--', color='k')

for i in range(2):
    ax[i].set_xticks(np.linspace(0, 360, 5))
    ax[i].axhline(0, ls='--', color='k')
    ax[i].set_xlabel("Pref. Location (°)")
    ax[i].set_ylim([-1.25, 1.25])

plt.savefig('./figures/bernstein/test_weights.svg', dpi=300)
plt.show()
```

Go/NoGo Task
============

Test
----

``` ipython
model_state_dict = torch.load('%s/dual_naive_%d.pth' % (path, seed))
model.load_state_dict(model_state_dict, strict=False)
model.eval();
```

``` ipython
model.N_BATCH = 1

model.I0[0] = 0
model.I0[1] = 1.0
model.I0[2] = 1.0
model.I0[3] = 0.0
model.I0[4] = 0.0

A = model.init_ff_input()

model.I0[0] = 0 # NoGo
model.I0[1] = -1.0 # cue
model.I0[2] = 1.0
model.I0[3] = 0.0
model.I0[4] = 0.0

B = model.init_ff_input()

ff_input = torch.cat((A, B))
print('ff_input', ff_input.shape)
```

``` example
ff_input torch.Size([2, 605, 1000])
```

``` ipython
with torch.no_grad():
    rates = model.forward(ff_input=ff_input).cpu().detach().numpy()
print(rates.shape)
```

``` ipython
plot_overlap(model.readout.cpu().detach().numpy(), labels=['Go', 'NoGo'], figname='../figures/dual/GoNoGo_overlaps_%d.svg' % seed)
```

``` ipython
U = model.low_rank.U.cpu().detach().numpy()
V = model.low_rank.V.cpu().detach().numpy()

fig, ax = plt.subplots(1, 2, figsize=[2*width, height])
# ax[0].hist(U[:, 0], histtype='step', bins='auto')
# ax[0].hist(U[:, 1], histtype='step', bins='auto')
ax[0].hist(V[:, 0], histtype='step', bins='auto')
ax[1].hist(V[:, 1], histtype='step', bins='auto')
ax[0].set_xlabel('$ n_{AB} $')
ax[1].set_xlabel('$ n_{GNG} $')

ax[0].set_ylabel('Count')
ax[1].set_ylabel('Count')
plt.show()
```

``` ipython
U = model.low_rank.U.cpu().detach().numpy()
V = model.low_rank.V.cpu().detach().numpy()

fig, ax = plt.subplots(1, 4, figsize=[4*height, height])

ax[0].scatter(U[:, 0], U[:, 1])
ax[0].set_xlabel('$ m_{AB} $')
ax[0].set_ylabel('$ m_{Choice} $')

ax[1].scatter(V[:, 0], V[:, 1])
ax[1].set_xlabel('$ n_{AB} $')
ax[1].set_ylabel('$ n_{Choice} $')

ax[2].scatter(U[:, 0], V[:, 0])
ax[2].set_xlabel('$ m_{AB} $')
ax[2].set_ylabel('$ n_{AB} $')

ax[3].scatter(U[:, 1], V[:, 1])
ax[3].set_xlabel('$ m_{Choice} $')
ax[3].set_ylabel('$ n_{Choice} $')

plt.savefig('lr_scatter_dual_naive.svg', dpi=300)
plt.show()
```

``` ipython
U = model.low_rank.U.cpu().detach().numpy()[model.slices[0], 0]
V = model.low_rank.V.cpu().detach().numpy()[model.slices[0], 0]

odors = model.odors.cpu().numpy()

m = model.low_rank.U.cpu().detach().numpy()[model.slices[0], 1]
n = model.low_rank.V.cpu().detach().numpy()[model.slices[0], 1]

vectors = [U, V, m, n]
labels = ['$m_\\text{AB}$', '$n_\\text{AB}$', '$m_\\text{GnG}$', '$n_\\text{GnG}$']

import numpy as np
import matplotlib.pyplot as plt

# Calculate the covariance matrix
num_vectors = len(vectors)
cov_matrix = np.zeros((num_vectors, num_vectors))

for i in range(num_vectors):
    for j in range(num_vectors):
        cov_matrix[i][j] = angle_AB(vectors[i], vectors[j])

# Mask the upper triangle
mask = np.triu(np.ones_like(cov_matrix, dtype=bool))
masked_cov_matrix = np.ma.masked_array(cov_matrix, mask=mask)

plt.figure(figsize=(8, 6))

# Plot the masked covariance matrix
img = plt.imshow(masked_cov_matrix, cmap=custom_cmap, interpolation='nearest', vmin=30, vmax=150)
cbar = plt.colorbar(label='Angle (°)')
cbar.set_ticks([30, 90, 120])

# Set axis labels on top and left
# plt.gca().xaxis.tick_top()
plt.xticks(ticks=np.arange(num_vectors), labels=labels)
plt.yticks(ticks=np.arange(num_vectors), labels=labels)

# Invert y-axis
plt.gca().invert_yaxis()

for i in range(num_vectors):
    for j in range(i + 1):
        plt.text(j, i, f'{cov_matrix[i, j]:.0f}', ha='center', va='center', color='black')

plt.savefig('../figures/dual/cov_drt_%d.svg' % seed, dpi=300)
plt.show()
```

``` ipython
```

Dual Task
=========

Testing
-------

### Simulations

``` ipython
model_state_dict = torch.load('%s/dual_naive_%d.pth' % (path, seed))
model.load_state_dict(model_state_dict, strict=False)
model.eval();
```

``` ipython
steps = np.arange(0, model.N_STEPS - model.N_STEADY, model.N_WINDOW)

mask_rwd = (steps >= (model.N_STIM_ON[-1].cpu().numpy() - model.N_STEADY))
rwd_idx = np.where(mask_rwd)[0]
print('rwd', rwd_idx)

mask_cue = (steps >= (model.N_STIM_ON[2].cpu().numpy() - model.N_STEADY)) & (steps <= (model.N_STIM_OFF[3].cpu().numpy() - model.N_STEADY))
cue_idx = np.where(mask_cue)[0]
print('cue', cue_idx)

mask_GnG = (steps >= (model.N_STIM_OFF[1].cpu().numpy() - model.N_STEADY)) & (steps <= (model.N_STIM_ON[2].cpu().numpy() - model.N_STEADY))
GnG_idx = np.where(mask_GnG)[0]
print('GnG', GnG_idx)

mask_stim = (steps >= (model.N_STIM_ON[0].cpu().numpy() - model.N_STEADY)) & (steps <= (model.N_STIM_ON[-1].cpu().numpy() - model.N_STEADY))
stim_idx = np.where(mask_stim)[0]
print('stim', stim_idx)

mask_zero = ~mask_rwd & ~mask_cue & ~mask_stim
zero_idx = np.where(mask_zero)[0]
print('zero', zero_idx)
```

``` ipython
U = model.low_rank.U.cpu().detach().numpy()[model.slices[0], 0]
V = model.low_rank.V.cpu().detach().numpy()[model.slices[0], 0]

odors = model.odors.cpu().numpy()

m = model.low_rank.U.cpu().detach().numpy()[model.slices[0], 1]
n = model.low_rank.V.cpu().detach().numpy()[model.slices[0], 1]

vectors = [U, V, m, n]
labels = ['$m_\\text{AB}$', '$n_\\text{AB}$', '$m_\\text{GnG}$', '$n_\\text{GnG}$']

import numpy as np
import matplotlib.pyplot as plt

# Calculate the covariance matrix
num_vectors = len(vectors)
cov_matrix = np.zeros((num_vectors, num_vectors))

for i in range(num_vectors):
    for j in range(num_vectors):
        cov_matrix[i][j] = angle_AB(vectors[i], vectors[j])

# Mask the upper triangle
mask = np.triu(np.ones_like(cov_matrix, dtype=bool))
masked_cov_matrix = np.ma.masked_array(cov_matrix, mask=mask)

#+RESULTS:

plt.figure(figsize=(8, 6))

# Plot the masked covariance matrix
img = plt.imshow(masked_cov_matrix, cmap=custom_cmap, interpolation='nearest', vmin=30, vmax=150)
cbar = plt.colorbar(label='Angle (°)')
cbar.set_ticks([30, 90, 120])

# Set axis labels on top and left
# plt.gca().xaxis.tick_top()
plt.xticks(ticks=np.arange(num_vectors), labels=labels)
plt.yticks(ticks=np.arange(num_vectors), labels=labels)

# Invert y-axis
plt.gca().invert_yaxis()

for i in range(num_vectors):
    for j in range(i + 1):
        plt.text(j, i, f'{cov_matrix[i, j]:.0f}', ha='center', va='center', color='black')

plt.savefig('../figures/dual/cov_naive_%d.svg' % seed, dpi=300)
plt.show()
```

``` ipython
N_BATCH = 64
model.N_BATCH = N_BATCH

model.lr_eval_win = np.max( (rwd_idx.shape[0], cue_idx.shape[0]))

ff_input = []
labels = np.zeros((3, 12, model.N_BATCH, model.lr_eval_win))

l=0
for j in [0, 1, -1]:
    for i in [-1, 1]:
        for k in [-1, 1]:

            model.I0[0] = i # sample
            model.I0[1] = j # distractor
            model.I0[4] = k # test

            if i==1:
                labels[1, l] = np.ones((model.N_BATCH, model.lr_eval_win))

            if i==k: # Pair Trials
                labels[0, l] = np.ones((model.N_BATCH, model.lr_eval_win))

            if j==1: # Go
                model.I0[2] = 1.0 # cue
                model.I0[3] = 0.0

                labels[2, l] = np.ones((model.N_BATCH, model.lr_eval_win))

            elif j==-1: # NoGo
                model.I0[2] = 1.0 # cue
                model.I0[3] = 0.0 # rwd
                labels[2, l] = -np.ones((model.N_BATCH, model.lr_eval_win))

            else: # DPA
                model.I0[2] = 0 # cue
                model.I0[3] = 0 # rwd

            l+=1

            ff_input.append(model.init_ff_input())


labels = torch.tensor(labels, dtype=torch.float, device=DEVICE).reshape(3, -1, model.lr_eval_win).transpose(0, 1)

ff_input = torch.vstack(ff_input)
print('ff_input', ff_input.shape, 'labels', labels.shape)
```

``` ipython
with torch.no_grad():
    rates = model.forward(ff_input=ff_input).cpu().detach().numpy()
print(rates.shape)
```

``` ipython
import numpy as np

X1 = model.odors[4].cpu().numpy()  # class 1 samples
X2 = -model.odors[9].cpu().numpy()  # class 2 samples

w = X2 - X1                # normal vector to the hyperplan
c = 0.5 * np.dot(w, X2 + X1)  # offset

def predict(x):
    return (x @ w ) / w.shape[0] # >0: side of X1, <0: side of X2
```

``` ipython
naive_pred = predict(rates)
```

``` example
torch.Size([768, 3, 31])
```

``` ipython
naive_labels = labels.cpu().numpy().copy()
idxPair = (naive_labels[:, 0, 0]==1) & (naive_labels[:, 2, 0]==0)
idxUnpair = (naive_labels[:, 0, 0]==0) & (naive_labels[:, 2, 0]==0)
```

``` ipython
plt.plot(np.linspace(0, 10, 101), ((2 * naive_labels[:, 0, 0]-1)[idxPair, None] * naive_pred[idxPair]).mean(0), label='Pair', color='r')
plt.plot(np.linspace(0, 10, 101), ((2 * naive_labels[:, 0, 0]-1)[idxUnpair, None] * naive_pred[idxUnpair]).mean(0), label='Unpair', color='r', ls='--')

add_vlines()
plt.xlabel('Time')
plt.ylabel('Test Overlap')
plt.ylim([-1, 1])
plt.axhline(0, ls='--')
plt.legend(fontsize=12, frameon=0)
plt.savefig('../figures/bernstein/dpa_test_overlap.svg', dpi=300)
plt.show()
```

``` ipython
U = model.low_rank.U.cpu().detach().numpy()
V = model.low_rank.V.cpu().detach().numpy()

theta = np.arctan2(V[:, 1], V[:, 0])
idx = np.argsort(theta)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

ax[0].plot(np.linspace(0, 360, 750), circcvl(model.odors[0][idx].cpu(), windowSize=100), label='A')
ax[0].plot(np.linspace(0, 360, 750), -circcvl(model.odors[5][idx].cpu(), windowSize=100), label='B')

ax[0].set_xlabel('Pref Loc. (°)')
ax[0].set_ylabel('Sample Input')
ax[0].set_xticks(np.linspace(0, 360, 5))
ax[0].legend(fontsize=12, frameon=False)

ax[1].plot(np.linspace(0, 360, 750), circcvl(model.odors[4][idx].cpu(), windowSize=100), label='C')
ax[1].plot(np.linspace(0, 360, 750), -circcvl(model.odors[9][idx].cpu(), windowSize=100), label='D')

ax[1].set_xlabel('Pref Loc. (°)')
ax[1].set_ylabel('Test Input')
ax[1].set_xticks(np.linspace(0, 360, 5))
ax[1].legend(fontsize=12, frameon=False)

plt.savefig('../figures/bernstein/dual_naive_inputs.svg', dpi=300)

plt.show()
```

### Overlaps

``` ipython
readout = model.readout.cpu().detach().numpy()
y_labels = labels[..., 0].T.cpu().numpy().copy()
print(readout.shape, y_labels.shape)
```

``` ipython
def plot_overlap_label(readout, y, task=0, label=['pair', 'unpair'], figname='fig.svg', title='first'):
    '''
    y[0] is pairs, y[1] is samples, y[2] is task if not None
    '''
    print(y.shape)
    size = y.shape[0]
    if size ==2:
        ones_slice = np.zeros(y.shape)
        y_ = np.vstack((y.copy(), ones_slice))
        task = 0
    else:
        y_ = y.copy()
        tasks = [0, 1, -1]

    print(y_.shape)
    fig, ax = plt.subplots(1, 3, figsize=[3*width, height], sharey=True)

    time = np.linspace(0, 10, readout.shape[1])
    colors = ['r', 'b', 'g']
    ls = ['--', '-', '-', '--']
    label = ['AD', 'AC', 'BD', 'BC']
    mean_overlaps = []
    for k in range(2): # readout
        for j in range(2): # sample
            for i in range(2): # pair
                data = readout[(y_[0]==i) & (y_[1]==j) & (y_[2]==task), :, k]
                mean, ci = mean_ci(data)
                mean_overlaps.append(mean)
                ax[k].plot(time, mean, ls=ls[i+2*j], label=label[i+2*j], color=colors[task], alpha=1-(1-j)/2)
                ax[k].fill_between(time, mean - ci, mean + ci, color=colors[task], alpha=0.1)

        add_vlines(ax[k])
        ax[k].set_xlabel('Time (s)')

        if k==0:
            ax[k].set_ylabel('A/B Overlap (Hz)')
        elif k==1:
            ax[k].set_ylabel('GNG Overlap (Hz)')
        else:
            ax[k].set_ylabel('Readout (Hz)')

        ax[k].axhline(0, color='k', ls='--')

    mean_overlaps = np.array(mean_overlaps).reshape((2, 2, 2, -1))
    print(mean_overlaps.shape)
    for j in range(2): # sample
        for i in range(2): # pair
            ax[-1].plot(mean_overlaps[0, j, i], mean_overlaps[1, j, i], color=colors[task], ls=ls[i+2*j], label=label[i+2*j], alpha=1-(1-j)/2)

    ax[-1].set_xlabel('A/B Overlap (Hz)')
    ax[-1].set_ylabel('Choice Overlap (Hz)')

    plt.legend(fontsize=10)
    plt.savefig('../figures/dual/%s' % figname, dpi=300)
    plt.show()
```

``` ipython
plot_overlap_label(readout, y_labels, task=0, figname='overlaps_naive_dpa.svg')
```

``` ipython
plot_overlap_label(readout, y_labels, task=1, figname='overlaps_naive_go.svg')
```

``` ipython
plot_overlap_label(readout, y_labels, task=-1, figname='overlaps_naive_nogo.svg')
```

``` ipython
```

### Perf

``` ipython
from src.train.dual.task_score import *

criterion = DualScore(cue_idx=cue_idx, rwd_idx=rwd_idx, read_idx=[1, 1], DEVICE=DEVICE)
dpa_perf, drt_perf = criterion(model.readout, labels.clone())
```

``` ipython
dpa_mean = []
dpa_sem = []
for i in [0, 1, -1]:
        y = torch.where(labels[:, 2, 0]==i)

        mean_, sem_ = calculate_mean_accuracy_and_sem(dpa_perf[y])
        dpa_mean.append(mean_)
        dpa_sem.append(sem_)

dpa_mean = torch.stack(dpa_mean)
dpa_sem = np.stack(dpa_sem)
```

``` ipython
print(dpa_mean)
```

``` ipython
drt_mean, drt_sem = calculate_mean_accuracy_and_sem(drt_perf)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[1.5*width, height])

pal = ['r', 'b', 'g']
for i in range(3):
    ax[0].errorbar(i, dpa_mean[i].item(), yerr=dpa_sem[i].item(), fmt='o', label='Naive',
                   color=pal[i], ecolor=pal[i], elinewidth=3, capsize=5)

ax[0].set_xlim(-1, 4)
# ax[0].set_ylim(0.4, 1.1)

ax[0].set_ylabel('DPA Accuracy')
ax[0].set_xticks([1], ['Naive'])
ax[0].axhline(y=0.5, color='k', linestyle='--')

ax[1].errorbar(0, drt_mean.item(), yerr=drt_sem.item(), fmt='o', label='Naive',
             color='k', ecolor='k', elinewidth=3, capsize=5)

ax[1].set_xlim(-1, 2)
ax[1].set_ylim(0.4, 1.1)

ax[1].set_ylabel('Go/NoGo Accuracy')
ax[1].set_xticks([0, 1], ['Naive', 'Expert'])
ax[1].axhline(y=0.5, color='k', linestyle='--')

plt.savefig('../figures/dual/dual_perf_%d.svg' % seed, dpi=300)

plt.show()
```

``` ipython
U = model.low_rank.U.cpu().detach().numpy()
V = model.low_rank.V.cpu().detach().numpy()

fig, ax = plt.subplots(1, 4, figsize=[4*height, height])

ax[0].scatter(U[:, 0], U[:, 1])
ax[0].set_xlabel('$ m_{AB} $')
ax[0].set_ylabel('$ m_{Choice} $')

ax[1].scatter(V[:, 0], V[:, 1])
ax[1].set_xlabel('$ n_{AB} $')
ax[1].set_ylabel('$ n_{Choice} $')

ax[2].scatter(U[:, 0], V[:, 0])
ax[2].set_xlabel('$ m_{AB} $')
ax[2].set_ylabel('$ n_{AB} $')

ax[3].scatter(U[:, 1], V[:, 1])
ax[3].set_xlabel('$ m_{Choice} $')
ax[3].set_ylabel('$ n_{Choice} $')

plt.savefig('lr_scatter_dual_naive.svg', dpi=300)
plt.show()
```

``` ipython
```

Re-Testing
----------

### run

``` ipython
model_state_dict = torch.load('%s/dual_train_%d.pth' % (path, seed))
model.load_state_dict(model_state_dict, strict=False)
model.eval();
```

``` ipython
steps = np.arange(0, model.N_STEPS - model.N_STEADY, model.N_WINDOW)

mask_rwd = (steps >= (model.N_STIM_ON[-1].cpu().numpy() - model.N_STEADY))
rwd_idx = np.where(mask_rwd)[0]
print('rwd', rwd_idx)

mask_cue = (steps >= (model.N_STIM_ON[2].cpu().numpy() - model.N_STEADY)) & (steps <= (model.N_STIM_OFF[3].cpu().numpy() - model.N_STEADY))
cue_idx = np.where(mask_cue)[0]
print('cue', cue_idx)

mask_GnG = (steps >= (model.N_STIM_ON[1].cpu().numpy() - model.N_STEADY)) & (steps <= (model.N_STIM_ON[2].cpu().numpy() - model.N_STEADY))
GnG_idx = np.where(mask_GnG)[0]
print('GnG', GnG_idx)

mask_stim = (steps >= (model.N_STIM_ON[0].cpu().numpy() - model.N_STEADY)) & (steps <= (model.N_STIM_ON[-1].cpu().numpy() - model.N_STEADY))
stim_idx = np.where(mask_stim)[0]
print('stim', stim_idx)

mask_zero = ~mask_rwd & ~mask_cue & ~mask_stim
zero_idx = np.where(mask_zero)[0]
print('zero', zero_idx)
```

``` ipython
U = model.low_rank.U.cpu().detach().numpy()[model.slices[0], 0]
V = model.low_rank.V.cpu().detach().numpy()[model.slices[0], 0]

odors = model.odors.cpu().numpy()

m = model.low_rank.U.cpu().detach().numpy()[model.slices[0], 1]
n = model.low_rank.V.cpu().detach().numpy()[model.slices[0], 1]

vectors = [U, V, m, n]
labels = ['$m_\\text{AB}$', '$n_\\text{AB}$', '$m_\\text{GnG}$', '$n_\\text{GnG}$']

import numpy as np
import matplotlib.pyplot as plt

# Calculate the covariance matrix
num_vectors = len(vectors)
cov_matrix = np.zeros((num_vectors, num_vectors))

for i in range(num_vectors):
    for j in range(num_vectors):
        cov_matrix[i][j] = angle_AB(vectors[i], vectors[j])

# Mask the upper triangle
mask = np.triu(np.ones_like(cov_matrix, dtype=bool))
masked_cov_matrix = np.ma.masked_array(cov_matrix, mask=mask)

plt.figure(figsize=(8, 6))

# Plot the masked covariance matrix
img = plt.imshow(masked_cov_matrix, cmap=custom_cmap, interpolation='nearest', vmin=30, vmax=150)
cbar = plt.colorbar(label='Angle (°)')
cbar.set_ticks([30, 90, 120])

# Set axis labels on top and left
# plt.gca().xaxis.tick_top()
plt.xticks(ticks=np.arange(num_vectors), labels=labels)
plt.yticks(ticks=np.arange(num_vectors), labels=labels)

# Invert y-axis
plt.gca().invert_yaxis()

for i in range(num_vectors):
    for j in range(i + 1):
        plt.text(j, i, f'{cov_matrix[i, j]:.0f}', ha='center', va='center', color='black')
plt.savefig('../figures/dual/cov_train_%d.svg' % seed, dpi=300)
plt.show()
```

``` ipython
N_BATCH = 32
model.N_BATCH = N_BATCH

model.lr_eval_win = np.max( (rwd_idx.shape[0], cue_idx.shape[0]))

ff_input = []
labels = np.zeros((3, 12, model.N_BATCH, model.lr_eval_win))

l=0
for j in [0, 1, -1]:
    for i in [-1, 1]:
        for k in [-1, 1]:

            model.I0[0] = i # sample
            if i==1:
                labels[1, l] = i * np.ones((model.N_BATCH, model.lr_eval_win))

            model.I0[1] = j # distractor
            model.I0[4] = k # test

            if i==k: # Pair Trials
                labels[0, l] = np.ones((model.N_BATCH, model.lr_eval_win))

            if j==1: # Go
                model.I0[2] = 1.0 # cue
                model.I0[3] = 0.0

                labels[2, l] = np.ones((model.N_BATCH, model.lr_eval_win))
            elif j==-1: # NoGo
                model.I0[2] = 1.0 # cue
                model.I0[3] = 0.0 # rwd
                labels[2, l] = -np.ones((model.N_BATCH, model.lr_eval_win))
            else: # DPA
                model.I0[2] = 0 # cue
                model.I0[3] = 0 # rwd

            l+=1

            ff_input.append(model.init_ff_input())


labels = torch.tensor(labels, dtype=torch.float, device=DEVICE).reshape(3, -1, model.lr_eval_win).transpose(0, 1)

ff_input = torch.vstack(ff_input)
print('ff_input', ff_input.shape, 'labels', labels.shape)
```

``` ipython
with torch.no_grad():
    rates = model.forward(ff_input=ff_input).cpu().detach().numpy()
print(rates.shape)
```

``` ipython
train_pred = predict(rates)
```

``` ipython
train_labels = labels.cpu().numpy().copy()
idxPair = (train_labels[:, 0, 0]==1) & (train_labels[:, 2, 0]==0)
idxUnpair = (train_labels[:, 0, 0]==0) & (train_labels[:, 2, 0]==0)
```

``` ipython
plt.plot(np.linspace(0, 10, 101), ((2 * train_labels[:, 0, 0]-1)[idxPair, None] * train_pred[idxPair]).mean(0), label='Pair', color='r')
plt.plot(np.linspace(0, 10, 101), ((2 * train_labels[:, 0, 0]-1)[idxUnpair, None] * train_pred[idxUnpair]).mean(0), label='Unpair', color='r', ls='--')

add_vlines()
plt.xlabel('Time')
plt.ylabel('Test Overlap')
plt.ylim([-1, 1])
plt.axhline(0, ls='--')
plt.legend(fontsize=12, frameon=0)
plt.savefig('../figures/bernstein/dpa_test_overlap.svg', dpi=300)
plt.show()
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

idxPair = (naive_labels[:, 0, 0]==1) & (naive_labels[:, 2, 0]==0)
idxUnpair = (naive_labels[:, 0, 0]==0) & (naive_labels[:, 2, 0]==0)

ax[0].plot(np.linspace(0, 10, 101), ((2 * naive_labels[:, 0, 0]-1)[idxPair, None] * naive_pred[idxPair]).mean(0), label='Naive Pair', color='r', alpha=0.5, ls='--')
ax[1].plot(np.linspace(0, 10, 101), ((2 * naive_labels[:, 0, 0]-1)[idxUnpair, None] * naive_pred[idxUnpair]).mean(0), label='Naive Unpair', color='r', ls='--', alpha=0.5)

idxPair = (train_labels[:, 0, 0]==1) & (train_labels[:, 2, 0]==0)
idxUnpair = (train_labels[:, 0, 0]==0) & (train_labels[:, 2, 0]==0)

ax[0].plot(np.linspace(0, 10, 101), ((2 * train_labels[:, 0, 0]-1)[idxPair, None] * train_pred[idxPair]).mean(0), label='Expert Pair', color='r')
ax[1].plot(np.linspace(0, 10, 101), ((2 * train_labels[:, 0, 0]-1)[idxUnpair, None] * train_pred[idxUnpair]).mean(0), label='Expert Unpair', color='r')

for i in range(2):
    add_vlines(ax[i])
    ax[i].set_xlabel('Time')
    ax[i].set_ylabel('Test Overlap')
    ax[i].set_ylim([-1, 1])
    ax[i].axhline(0, ls='--')
    ax[i].legend(fontsize=12, frameon=0)

plt.savefig('../figures/bernstein/dual_test_overlap.svg', dpi=300)
plt.show()
```

``` ipython
U = model.low_rank.U.cpu().detach().numpy()
V = model.low_rank.V.cpu().detach().numpy()

theta = np.arctan2(V[:, 1], V[:, 0])
idx = np.argsort(theta)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

ax[0].plot(np.linspace(0, 360, 750), circcvl(model.odors[0][idx].cpu(), windowSize=100), label='A')
ax[0].plot(np.linspace(0, 360, 750), -circcvl(model.odors[5][idx].cpu(), windowSize=100), label='B')

ax[0].set_xlabel('Pref Loc. (°)')
ax[0].set_ylabel('Sample Input')
ax[0].set_xticks(np.linspace(0, 360, 5))
ax[0].legend(fontsize=12, frameon=False)

ax[1].plot(np.linspace(0, 360, 750), circcvl(model.odors[4][idx].cpu(), windowSize=100), label='C')
ax[1].plot(np.linspace(0, 360, 750), -circcvl(model.odors[9][idx].cpu(), windowSize=100), label='D')

ax[1].set_xlabel('Pref Loc. (°)')
ax[1].set_ylabel('Test Input')
ax[1].set_xticks(np.linspace(0, 360, 5))
ax[1].legend(fontsize=12, frameon=False)

plt.savefig('../figures/bernstein/dual_train_inputs.svg', dpi=300)

plt.show()
```

### Overlaps

``` ipython
readout = model.readout.cpu().detach().numpy()
y_labels = labels[..., 0].T.cpu().numpy().copy()
print(readout.shape, y_labels.shape)
```

``` ipython
plot_overlap_label(readout, y_labels, task=0, figname='overlaps_expert_dpa.svg')
```

``` ipython
plot_overlap_label(readout, y_labels, task=1, figname='overlaps_expert_go.svg')
```

``` ipython
plot_overlap_label(readout, y_labels, task=-1, figname='overlaps_expert_nogo.svg')
```

``` ipython
```

### Perf

``` ipython
from src.train.dual.task_score import *

criterion = DualScore(cue_idx=cue_idx, rwd_idx=rwd_idx, read_idx=[1, 1], DEVICE=DEVICE)
dpa_perf, drt_perf = criterion(model.readout, labels.clone())
```

``` ipython
dpa_mean2 = []
dpa_sem2 = []
for i in [0, 1, -1]:
        y = torch.where(labels[:, 2, 0]==i)

        mean_, sem_ = calculate_mean_accuracy_and_sem(dpa_perf[y])
        dpa_mean2.append(mean_)
        dpa_sem2.append(sem_)

dpa_mean2 = torch.stack(dpa_mean2)
dpa_sem2 = np.stack(dpa_sem2)
```

``` ipython
drt_mean2, drt_sem2 = calculate_mean_accuracy_and_sem(drt_perf)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[1.5*width, height])

pal = ['r', 'b', 'g']
for i in range(3):
    ax[0].errorbar(i, dpa_mean[i].item(), yerr=dpa_sem[i].item(), fmt='o', label='Naive',
                   color=pal[i], ecolor=pal[i], elinewidth=3, capsize=5)

    ax[0].errorbar(i+4, dpa_mean2[i].item(), yerr=dpa_sem2[i].item(), fmt='o', label='Naive',
                   color=pal[i], ecolor=pal[i], elinewidth=3, capsize=5)

ax[0].set_xlim(-1, 7)
ax[0].set_ylim(0.4, 1.1)

ax[0].set_ylabel('DPA Accuracy')
ax[0].set_xticks([1, 5], ['Naive', 'Expert'])
ax[0].axhline(y=0.5, color='k', linestyle='--')

ax[1].errorbar(0, drt_mean.item(), yerr=drt_sem.item(), fmt='o', label='Naive',
             color='k', ecolor='k', elinewidth=3, capsize=5)
ax[1].errorbar(1, drt_mean2.item(), yerr=drt_sem2.item(), fmt='o', label='Naive',
             color='k', ecolor='k', elinewidth=3, capsize=5)

ax[1].set_xlim(-1, 2)
ax[1].set_ylim(0.4, 1.1)

ax[1].set_ylabel('Go/NoGo Accuracy')
ax[1].set_xticks([0, 1], ['Naive', 'Expert'])
ax[1].axhline(y=0.5, color='k', linestyle='--')

plt.savefig('../figures/dual/dual_perf_%d.svg' % seed, dpi=300)

plt.show()
```

``` ipython
U = model.low_rank.U.cpu().detach().numpy()
V = model.low_rank.V.cpu().detach().numpy()

fig, ax = plt.subplots(1, 2, figsize=[2*width, height])
# ax[0].hist(U[:, 0], histtype='step', bins='auto')
# ax[0].hist(U[:, 1], histtype='step', bins='auto')
ax[0].hist(V[:, 0], histtype='step', bins='auto')
ax[1].hist(V[:, 1], histtype='step', bins='auto')
ax[0].set_xlabel('$ n_{AB} $')
ax[1].set_xlabel('$ n_{GNG} $')

ax[0].set_ylabel('Count')
ax[1].set_ylabel('Count')
plt.show()
```

``` ipython
plt.scatter(V[:, 0], V[:, 1])
plt.xlabel('$ n_{AB} $')
plt.ylabel('$ n_{GNG} $')
plt.show()
```

``` ipython
U = model.low_rank.U.cpu().detach().numpy()
V = model.low_rank.V.cpu().detach().numpy()

fig, ax = plt.subplots(1, 4, figsize=[4*height, height])

ax[0].scatter(U[:, 0], U[:, 1])
ax[0].set_xlabel('$ m_{AB} $')
ax[0].set_ylabel('$ m_{Choice} $')

ax[1].scatter(V[:, 0], V[:, 1])
ax[1].set_xlabel('$ n_{AB} $')
ax[1].set_ylabel('$ n_{Choice} $')

ax[2].scatter(U[:, 0], V[:, 0])
ax[2].set_xlabel('$ m_{AB} $')
ax[2].set_ylabel('$ n_{AB} $')

ax[3].scatter(U[:, 1], V[:, 1])
ax[3].set_xlabel('$ m_{Choice} $')
ax[3].set_ylabel('$ n_{Choice} $')

plt.savefig('lr_scatter_dual_train.svg', dpi=300)
plt.show()
```

``` ipython
```